In [28]:
import tensorflow as tf
from tensorflow.keras import Sequential
import tensorflow.keras.layers as lay
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [29]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [30]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [31]:
X_train.shape

(60000, 28, 28)

In [32]:
X_train = X_train.reshape((X_train.shape[0],28,28,1))
X_test = X_test.reshape((X_test.shape[0],28,28,1))

In [33]:
X_train.shape

(60000, 28, 28, 1)

In [34]:
X_test.shape

(10000, 28, 28, 1)

In [35]:
X_train = X_train.astype("float") /255.0
X_test = X_test.astype("float") / 255.0

In [36]:
X_train[0]

array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        

In [37]:
labels = LabelBinarizer()
y_train = labels.fit_transform(y_train)
y_test = labels.transform(y_test)

In [38]:
X_train.shape

(60000, 28, 28, 1)

In [39]:
y_train.shape

(60000, 10)

In [40]:
classifier = Sequential()
classifier.add(lay.Conv2D(32, (5,5), padding="same", activation = 'relu', input_shape=(28,28,1)))
classifier.add(lay.Conv2D(32, (5,5), padding="same", activation = 'relu'))
classifier.add(lay.MaxPooling2D(pool_size=(2,2)))
classifier.add(lay.Dropout(0.25))

classifier.add(lay.Conv2D(64, (3,3), padding="same", activation = 'relu'))
classifier.add(lay.Conv2D(32, (5,5), padding="same", activation = 'relu'))
classifier.add(lay.MaxPooling2D(pool_size=(2,2), strides=(2,2)))
classifier.add(lay.Dropout(0.25))
               
classifier.add(lay.Flatten())
classifier.add(lay.Dense(256, activation = "relu"))
classifier.add(lay.Dropout(0.5))
classifier.add(lay.Dense(10, activation = "softmax"))


In [41]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001, rho = 0.9, epsilon = 1e-08)

In [42]:
classifier.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics=["accuracy"])

In [46]:
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [47]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10, 
        zoom_range = 0.1,  
        width_shift_range=0.1,  
        height_shift_range=0.1)

datagen.fit(X_train)

In [48]:
batch_size = 86

model = classifier.fit(datagen.flow(X_train,y_train, batch_size=batch_size),
                                 validation_data=(X_test,y_test), 
                                 epochs=30,
                                 verbose = 2,
                                 steps_per_epoch = X_train.shape[0] // batch_size,
                                 callbacks=[learning_rate_reduction])

Epoch 1/30
697/697 - 195s - loss: 0.0632 - accuracy: 0.9818 - val_loss: 0.0210 - val_accuracy: 0.9930
Epoch 2/30
697/697 - 240s - loss: 0.0597 - accuracy: 0.9837 - val_loss: 0.0225 - val_accuracy: 0.9925
Epoch 3/30
697/697 - 230s - loss: 0.0583 - accuracy: 0.9833 - val_loss: 0.0192 - val_accuracy: 0.9941
Epoch 4/30
697/697 - 251s - loss: 0.0577 - accuracy: 0.9835 - val_loss: 0.0215 - val_accuracy: 0.9945
Epoch 5/30
697/697 - 232s - loss: 0.0574 - accuracy: 0.9842 - val_loss: 0.0191 - val_accuracy: 0.9952
Epoch 6/30
697/697 - 249s - loss: 0.0579 - accuracy: 0.9844 - val_loss: 0.0229 - val_accuracy: 0.9933
Epoch 7/30
697/697 - 240s - loss: 0.0627 - accuracy: 0.9838 - val_loss: 0.0240 - val_accuracy: 0.9936
Epoch 8/30
697/697 - 242s - loss: 0.0586 - accuracy: 0.9847 - val_loss: 0.0253 - val_accuracy: 0.9935

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 9/30
697/697 - 239s - loss: 0.0452 - accuracy: 0.9874 - val_loss: 0.0208 - val_accuracy: 0.9946
E

In [50]:
predict = classifier.predict(X_test)
print(classification_report(y_test.argmax(axis=1), predict.argmax(axis=1), target_names = [str(x) for x in labels.classes_]))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      0.99      1.00      1135
           2       0.99      1.00      1.00      1032
           3       0.99      1.00      1.00      1010
           4       0.99      1.00      0.99       982
           5       1.00      0.99      1.00       892
           6       1.00      0.99      0.99       958
           7       1.00      1.00      1.00      1028
           8       1.00      1.00      1.00       974
           9       1.00      0.99      0.99      1009

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [49]:
classifier.save("digit_classifier", save_format = "h5")